____
__Universidad Tecnologica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521 - Turno Jueves a la noche__<br/>
__Clase_07: Árboles de decisión__<br/>
__Autor: Santiago Chas__
____

## Prediccion del precio de las propiedades

Vamos a utilizar la informacion de https://www.properati.com.ar/ correspondiente a Departamentos y PH de Capital Federal

#### Importamos librerias

In [ ]:
#pip install openpyxl

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configura la opción para desactivar la notación científica
pd.set_option('display.float_format', '{:.2f}'.format)


# Configurar opciones para poder visualizar todas las columnas
pd.set_option('display.max_columns', None)

In [ ]:
# Importamos el dataset e imprimimos algunos registros
df = pd.read_csv("./properati_capital.csv")
df.sample(4)

In [ ]:
# Vemos las dimensiones del dataset
df.shape

### EDA

#### Distribucion del precio

In [ ]:
# Crear un boxplot para la columna "price"

plt.figure(figsize=(8, 6))  # Tamaño opcional del gráfico
plt.boxplot(df['price'])
plt.title('Boxplot de Precio')
plt.ylabel('Precio')
plt.show()

In [ ]:
# Analizamos estadisticamente la columna precio
df.price.describe().T

Como vemos, tenemos valores atípicos.

Por este motivo vamos a evaluar los percentiles 99 y 1

In [ ]:
# Calcular el percentil 99 de la columna 'price'

percentil_99 = df['price'].quantile(0.99)
print(f"El valor del percentil 99 es: {percentil_99}")
percentil_01 = df['price'].quantile(0.01)
print(f"El valor del percentil 1 es: {percentil_01}")

In [ ]:
# Nos vamos a quedar con los registros que esten entre estos valores

df = df[(df['price'] > percentil_01) & (df['price'] < percentil_99)]
df.shape

In [ ]:
# Crear un boxplot para la columna "price"
plt.figure(figsize=(8, 6))  # Tamaño opcional del gráfico
plt.boxplot(df['price'])
plt.title('Boxplot de Precio')
plt.ylabel('Precio')
plt.show()

In [ ]:
df.price.describe().T

## EDA y Feature Engineering

El Feature Engineering es el conjunto de técnicas destinadas a transformar y preparar los datos antes de que sean utilizados por un modelo de aprendizaje automático. Su objetivo principal es mejorar la calidad y relevancia de las variables (features) para que el modelo pueda aprender patrones de manera más efectiva.
Este proceso incluye tareas que ya vimos, como:

- Estandarización o normalización de los datos numéricos.
- Codificación de variables categóricas (por ejemplo, One-Hot Encoding).

Pero también abarca otras estrategias más avanzadas, como:

- Creación de nuevas variables a partir de las existentes (feature creation).
- Selección de características relevantes (feature selection).
- Manejo de outliers y valores faltantes
- Transformaciones no lineales
- Incorporación de datos externas


En esta notebook vamos a explorar varias de estas técnicas.

Pueden leer más aca: 
- https://www.analyticsvidhya.com/blog/2021/10/a-beginners-guide-to-feature-engineering-everything-you-need-to-know/
- https://builtin.com/articles/feature-engineering

In [ ]:
# Imprimimos los nombres de variables
df.columns

In [ ]:
# Eliminamos las variables "Unnamed: 0" y "id" ya que son variables keys del dataset y no nos aportan informacion

ids_vars = ["Unnamed: 0", "id"] # creo una lista de las variables que decidimos eliminar
df.drop(ids_vars, axis=1,inplace= True) # el parametro inplace nos permite guardar los nuevos cambios dentro de la variable existente
df.shape

Veamos ahora los tipos de datos de estas columnas

In [ ]:
df.info()

Vemos que existen variables con muchos valores nulos.

Hagamos este análisis de una manera mas precisa.

In [ ]:
# Calculamos % los valores faltantes para cada una de las variables
miss = pd.DataFrame(df.isnull().mean(), columns=["Missing"])
perc_miss = miss.loc[miss.Missing > 0]
perc_miss.sort_values("Missing", ascending = False)

Observamos que las variables l6, l5, price_period y l4 contienen un alto % de valores faltantes.

Por este motivo vamos a eliminarlos de nuestro dataset

In [ ]:
# Repetimos el método de eliminar columnas

vars_nulls = ["l6", "l5", "l4", "price_period"]
df.drop(vars_nulls, axis = 1, inplace= True )
df.shape

In [ ]:
df.tail(3)

Sospechamos que existen columnas con un unico valor de dato.

Si es asi estas columnas no nos aportan informacion para nuestra futura prediccion

In [ ]:
# Contamos la cantidad de valores unicos que tiene cada columna

for x in df.columns:
    unique_vals = df[f"{x}"].nunique() # nunique() cuenta la cantidad de valores unicos que tiene una columna
    if unique_vals > 2:
        None
    else:
        print(f'Variable {x}:', unique_vals)

Podemos ver esto mismo usando el método value_counts()

In [ ]:
# Vemos los valores para property_type y para l1

df.l1.value_counts(dropna=False)

In [ ]:
df.property_type.value_counts(dropna=False)

Property_type tiene sentido que solo tenga dos valores posibles ya que son los tipos de propiedades que queremos predecir.

Pero el resto de las variables que tienen valores unicos las eliminaremos.

In [ ]:
unique_vars = ["ad_type", "l1", "l2", "currency", "operation_type"]
df.drop(unique_vars, axis = 1, inplace= True)
df.head()

Las variables "created_on", "start_date", "end_date" no son variables útiles para lo que queremos hacer --> las eliminamos

Se podrían crear variables muy relevantes para nuestro modelo a partir de la latitud y la longitud pero no las utilizaremos en nuestro entrenamiento.

Algunas de las cosas que se pueden hacer pueden hacer pueden encontrarlas en este post: https://python.plainenglish.io/how-to-use-geo-location-for-feature-engineering-using-near-by-points-of-interest-563752e6ad26

In [ ]:
drop_vars = ["created_on", "start_date", "end_date", "lat", "lon"]
df.drop(drop_vars, axis=1, inplace= True)
df.shape

#### Enriquecimiento del dataset con una fuente externa

Importamos un dataset que extraemos de https://mudafy.com.ar/

Este contiene la informacion del precio promedio de m2 por barrio y la Comuna que corresponde para cada barrio

In [ ]:
df_m2 = pd.read_excel("./precio_prom_m2.xlsx")

In [ ]:
df_m2.head()

In [ ]:
df_m2.info(verbose=True)

In [ ]:
# Joineamos los dos df para traer las nuevas variables que seran de utildiad

df_merged = pd.merge(df, df_m2, left_on = "l3", right_on = "Barrio", how= 'left')
df_merged.drop(["Barrio"], axis=1, inplace=True)  # Eliminamos la variable Barrio ya que nos proporciona el mismo tipo de info que l3
df_merged.rename(columns={"Valor m2 (USD)":"valor_m2"}, inplace=True) # Renombramos la variable por un nombre mas sencillo
df_merged.shape

In [ ]:
df_merged.drop(["Comuna"], axis=1, inplace=True)

#### Tratamiento de valores faltantes

Volvemos hacer el análisis de missings para ver que tan poblada estan las nuevas variables

In [ ]:
miss = pd.DataFrame(df_merged.isnull().mean(), columns=["Missing"])
perc_miss = miss.loc[miss.Missing > 0]
perc_miss.sort_values("Missing", ascending = False)

In [ ]:
# Analizamos algunos valores estadísticos de las variables "bathrooms", "rooms", "bedrooms", "surface_total", "surface_covered", "valor_m2"

df_merged[["bathrooms", "rooms", "bedrooms", "surface_total", "surface_covered", "valor_m2"]].describe().T

Vemos que existen outliers para algunas de estas columnas, pero esto lo analizaremos luego.

Ahora nos concentraremos en la imputacion de valores faltantes aplicando distintas estrategias de imputacion segun la variable

In [ ]:
# Para bathrooms, room y bedrooms --> imputaremos con la mediana ya que deberian ser valores enteros


# Calculamos la media de cada columna
median_bathrooms = df_merged['bathrooms'].median()
median_rooms = df_merged['rooms'].median()
median_bedrooms = df_merged['bedrooms'].median()

# Imprime las medias
print(f"Mediana de 'bathrooms': {median_bathrooms}")
print(f"Mediana de 'rooms': {median_rooms}")
print(f"Mediana de 'bedrooms': {median_bedrooms}")


In [ ]:
# Imputamos los valores

df_merged['bathrooms'].fillna(median_bathrooms, inplace=True)
df_merged['rooms'].fillna(median_rooms, inplace=True)
df_merged['bedrooms'].fillna(median_bedrooms, inplace=True)

In [ ]:
# Como surface_total, surface_covered y valor_m2 pueden tomar valores decimales utilizamos la media para imputar sus nulos

# Calculamos las medias para cada columna
mean_surface_total = df_merged['surface_total'].mean()
mean_surface_covered = df_merged['surface_covered'].mean()
mean_valor_m2 = df_merged['valor_m2'].mean()

# Imprimimos las medias
print(f"Media de 'surface_total': {mean_surface_total}")
print(f"Media de 'surface_covered': {mean_surface_covered}")
print(f"Media de 'valor_m2': {mean_valor_m2}")

In [ ]:
# Imputamos los valores

df_merged['surface_total'].fillna(mean_surface_total, inplace=True)
df_merged['surface_covered'].fillna(mean_surface_covered, inplace=True)
df_merged['valor_m2'].fillna(mean_valor_m2, inplace=True)

#### Imputacion de missings para variables categoricas

In [ ]:
df_merged['l3'].fillna('sin_valor', inplace=True)


In [ ]:
# Verificamos los valores faltantes

df_merged.isnull().sum()

#### Tratamiento de valores atípicos/outliers

In [ ]:
# Volvemos a imprimir algunas metricas estadísticas para "bathrooms", "rooms", "bedrooms", "surface_total", "surface_covered"

df_merged[["bathrooms", "rooms", "bedrooms", "surface_total", "surface_covered"]].describe().T

Vamos a utilizar la tecnica de capping para no perder registros para nuestro entrenamiento

In [ ]:
# Capping superior
# Lista de columnas a procesar
cap_sup_cols = ["bathrooms", "rooms", "bedrooms", "surface_total", "surface_covered"]

# Iteramos a través de las columnas y reemplazamos los valores que superan el percentil 99
for columna in cap_sup_cols:
    percentil_99 = np.percentile(df_merged[columna], 99)
    df_merged[columna] = np.where(df_merged[columna] > percentil_99, percentil_99, df_merged[columna])

df_merged[["bathrooms", "rooms", "bedrooms", "surface_total", "surface_covered"]].describe().T

In [ ]:
# Capping inferior

cap_inf_cols = ["surface_total", "surface_covered"]

# Itera a través de las columnas y reemplaza los valores según la condición
for columna in cap_inf_cols:
    percentil_1 = np.percentile(df_merged[columna], 1)
    df_merged[columna] = np.where(df_merged[columna] < percentil_1, percentil_1, df_merged[columna])

df_merged[["bathrooms", "rooms", "bedrooms", "surface_total", "surface_covered"]].describe().T

#### Búsqueda de keywords en variables de texto

In [ ]:
# Concatenamos las variables del titulo y la descripcion para trabajar con una unica variable

df_merged['title_descr'] = df_merged['description'] + ' ' + df_merged['title']
df_merged.drop(["description", "title"], axis=1, inplace=True) # eliminamos las variables originales

Utilizaremos la libreria regex que es de expresiones regulares.

Documentacion: https://docs.python.org/3/library/re.html

In [ ]:
# Importamos la libreria re (regex)
import re

# Definimos las palabras clave
keywords = ["sum", "pileta", "amenities", "cochera"]

# Crea una columna para cada palabra clave y asigna 0 como valor inicial
for keyword in keywords:
    df_merged[keyword] = 0

# Itera a través de las palabras clave y asigna 1 si se encuentra la palabra clave en 'title_descr'
for keyword in keywords:
    # La siguiente línea crea un patrón de búsqueda de expresión regular (regex) para la palabra clave actual.
    # - El patrón \b{}\\b busca la palabra clave completa como una palabra independiente.
    # - El flag re.IGNORECASE hace que la búsqueda sea insensible a mayúsculas y minúsculas.
    pattern = re.compile(r'\b{}\b'.format(keyword), flags=re.IGNORECASE) #
    df_merged[keyword] = df_merged['title_descr'].apply(lambda x: 1 if pattern.search(str(x)) else 0)

In [ ]:
df_merged.shape

In [ ]:
# Iteramos a través de las columnas creadas y realiza un value_counts para cada una

for keyword_column in keywords:
    value_counts_keyword = df_merged[keyword_column].value_counts()
    print(f"Value Counts para '{keyword_column}':")
    print(value_counts_keyword)
    print()

In [ ]:
# Ahora que tenemos nuestras columnas creadas podemos eliminar la columna de texto
df_merged.drop(["title_descr"], axis= 1, inplace=True)

In [ ]:

df_merged.shape

In [ ]:
# Eliminamos la columna de barrios porque tienen muchos valores posibles. 
# TAREA: Agrupar los barrios bajo algun criterio de no mas de 5 valores posibles y ejecutar los modelos para ver si mejoran las predicciones
df_merged.drop(["l3"], axis=1, inplace=True)

#### Variables Categóricas

In [ ]:
y = df_merged[["price"]]
x = df_merged.drop(['price'], axis=1)

In [ ]:
x.info()

In [ ]:
x = pd.get_dummies(data=x, columns=['property_type'])
x.head()

In [ ]:
x.shape

### Machine Learning

In [ ]:
# Importamos librerias de Machine Learning


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV # Hace una busqueda de una cantidad determinada de parametros en lugar de todas las combinaciones posibles

# Documentacion de RandonSearchCV: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html


In [ ]:
# Train Test Split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=42)

#### Escalo la data

In [ ]:
scaler = preprocessing.StandardScaler().fit(xtrain)
xtrain_scal = scaler.transform(xtrain)
xtest_scal = scaler.transform(xtest)

#### Linear Regression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(xtrain_scal, ytrain)
ypred = lr.predict(xtest_scal)

In [ ]:
np.sqrt(mean_squared_error(ytest, ypred))

#### KNN Regressor

In [ ]:
knn = KNeighborsRegressor(weights = "distance")
parameters_k = np.arange(20,41,5)
parameters_knn = [{'n_neighbors': parameters_k}]
regressor_knn = GridSearchCV(knn, parameters_knn, refit = True, cv=5, scoring="neg_mean_squared_error", verbose=True)

In [ ]:
regressor_knn.fit(xtrain_scal, ytrain)

In [ ]:
regressor_knn.best_params_

In [ ]:
regressor_knn.best_score_

In [ ]:
ypred2 = regressor_knn.predict(xtest_scal)
np.sqrt(mean_squared_error(ytest, ypred2))

### Ensamble de árboles

#### Random Forest

In [ ]:
# Recuperar nombres de columnas desde xtrain
column_names = xtrain.columns


# Convertir arrays escalados en DataFrames con nombres de columnas
xtrain_scal_df = pd.DataFrame(xtrain_scal, columns=column_names)
xtest_scal_df = pd.DataFrame(xtest_scal, columns=column_names)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Definir el modelo base
rf = RandomForestRegressor(random_state=42)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=1
)

# Entrenar el modelo
random_search.fit(xtrain_scal_df, ytrain)

# Evaluar en el conjunto de test
best_model_rf = random_search.best_estimator_
y_pred = best_model_rf.predict(xtest_scal)
rmse = np.sqrt(mean_squared_error(ytest, y_pred))

print("Best Parameters:", random_search.best_params_)
print("Test RMSE:", rmse)


Los modelos de árboles nos permiten ver la importancia de las variables. Vamos a analizarlo

In [ ]:
# Obtener nombres de las variables desde el DataFrame con columnas
feature_names = xtrain_scal_df.columns

# Obtener importancia de variables
rf_importance = best_model_rf.feature_importances_

# Crear DataFrame ordenado
df_rf = pd.DataFrame({'Feature': feature_names, 'Importance': rf_importance}).sort_values(by='Importance', ascending=False)

In [ ]:
df_rf

#### XGBoost

In [ ]:
#pip install xgboost

In [ ]:
import xgboost as xgb

In [ ]:
# Definir el modelo base
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=1  # evitar problemas en algunos entornos
)

# Entrenar el modelo
random_search.fit(xtrain_scal, ytrain)

# Evaluar en el conjunto de test
best_model_xgb = random_search.best_estimator_
y_pred = best_model_xgb.predict(xtest_scal)
rmse = np.sqrt(mean_squared_error(ytest, y_pred))

print("Best Parameters:", random_search.best_params_)
print("Test RMSE:", rmse)

In [ ]:
# Obtener nombres de las variables desde el DataFrame con columnas
feature_names = xtrain_scal_df.columns

# Obtener importancia de variables
xgb_importance = best_model_xgb.feature_importances_

# Crear DataFrame ordenado
df_xgb = pd.DataFrame({'Feature': feature_names, 'Importance': xgb_importance}).sort_values(by='Importance', ascending=False)
df_xgb

#### LightGBM

In [ ]:
#pip install lightgbm

In [ ]:
import lightgbm as lgb

In [ ]:
# Definir el modelo base
lgbm = lgb.LGBMRegressor(random_state=42)

# Espacio de búsqueda de hiperparámetros
param_dist = {
    'num_leaves': [31, 50, 70],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=1  # evitar problemas en algunos entornos
)

# Entrenar el modelo
random_search.fit(xtrain_scal, ytrain)

# Evaluar en el conjunto de test
best_model_lgbm = random_search.best_estimator_
y_pred = best_model_lgbm.predict(xtest_scal)
rmse = np.sqrt(mean_squared_error(ytest, y_pred))

print("Best Parameters:", random_search.best_params_)
print("Test RMSE:", rmse)

In [ ]:
# Obtener nombres de las variables desde el DataFrame con columnas
feature_names = xtrain_scal_df.columns

# Obtener importancia de variables
lgbm_importance = best_model_lgbm.feature_importances_

# Crear DataFrame ordenado
df_lgbm = pd.DataFrame({'Feature': feature_names, 'Importance': lgbm_importance}).sort_values(by='Importance', ascending=False)

# Normalizar importancia en porcentaje
df_lgbm['Importance (%)'] = 100 * df_lgbm['Importance'] / np.sum(df_lgbm['Importance'])
df_lgbm

#### Tarea: Evaluar y probar que manera se pueden mejorar las performances de los modelos agregando nuevas variables y evaluando otros hiperparámetros